Author:
        
        PARK, JunHo, junho@ccnets.org

    COPYRIGHT (c) 2024. CCNets. All Rights reserved.

# Cheat Your Decision-Making Model: Manipulating Credit Card Data with Causal Inference

## Introduction

In this tutorial, we delve into the intricate world of causal inference to manipulate decision-making models in the domain of credit card fraud detection. Utilizing a Cooperative Network (CCNet), we explore how subtle manipulations in non-fraudulent data can expose and test the vulnerabilities of machine learning models. This demonstration not only highlights the robustness and weaknesses of current fraud detection systems but also illustrates the ethical considerations and potential risks inherent in data manipulation.


## Tutorial Goals

This tutorial provides a guide on using causal inference to manipulate data and evaluate its impact on machine learning models. We will cover:

1. **Training a Binary Classifier**: Train a model to distinguish between fraudulent and non-fraudulent transactions using real credit card data.
2. **Generating Manipulated Data with CCNet**: Use CCNet's Explainer, Reasoner, and Producer networks to create data that mimics non-fraudulent transactions but is designed to deceive the classifier by switching decision label factors.
3. **Evaluating Classifier Performance**: Test the classifier on both original and manipulated datasets to assess robustness. The classifier should perform well on both the original test set and the CCNet-recreated test set, indicating that CCNet has learned the data distribution that the classifier mapped for decision making.
4. **Comparative Dataset Analysis**:
   - **Testset A**: Real Test Data - Authentic credit card transactions used as a baseline to evaluate the classifier's performance.
   - **Testset B**: CCNet-generated data with random labels but maintaining the same frequency distribution as the original. The classifier's performance should be similar to Testset A, showing that CCNet understands the classifier's decision-making process.
   - **Testset C**: CCNet-generated data with original labels. With Testset B, it is proved that CCNet creates the genuine data distribution for any case of decision label. However, as most information of the data is in the explanation vector, it is easy to manipulate by changing the decision factor of the fraud data. In short, it is easy to manipulate the decision-making process by causal inference.
5. **Ethical Considerations**: Discuss the ethical implications of data manipulation in machine learning, emphasizing the need for robustness in model training and the risks of misuse in applications like fraud detection.


In [1]:
import sys
path_append = "../"
sys.path.append(path_append)  # Go up one directory from where you are.


In [2]:
# https://fraud-detection-handbook.github.io/fraud-detection-handbook/Chapter_7_DeepLearning/FeedForwardNeuralNetworks.html
import pandas as pd 

dataroot = path_append + "../data/credit_card_fraud_detection/creditcard.csv"
df = pd.read_csv(dataroot)
df

c:\Users\CCNets-team\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
c:\Users\CCNets-team\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


In [3]:

print('No Frauds', round(df['Class'].value_counts()[0] / len(df) *100,2), '%of the dataset')
print('Frauds', round(df['Class'].value_counts()[1] / len(df) *100,2), '%of the dataset')

No Frauds 99.83 %of the dataset
Frauds 0.17 %of the dataset


In [4]:
import torch
from sklearn.preprocessing import RobustScaler
from torch.utils.data import Dataset
# Initialize scalers
sc = RobustScaler()

# Scale all columns except the last one (which is the class column)
df.iloc[:, :-1] = sc.fit_transform(df.iloc[:, :-1])

# Calculate the number of features and classes
n_features = len(df.iloc[:, :-1].columns)
n_classes = len(df.iloc[:, -1:].columns)

print(n_features, n_classes)

30 1


In [5]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,-0.994983,-0.616237,-0.098602,1.228905,0.878152,-0.217859,0.631245,0.177406,0.142432,0.334787,...,0.026866,0.253109,-0.320791,0.032681,0.167619,-0.241182,0.816731,-0.246091,1.783274,0
1,-0.994983,0.524929,0.143100,-0.006970,0.293974,0.087726,0.164395,-0.105740,0.117064,-0.164482,...,-0.473332,-0.602719,0.363442,-0.479557,0.225462,0.313475,-0.063781,0.026519,-0.269825,0
2,-0.994972,-0.615587,-1.002407,0.830932,0.251024,-0.344345,1.778007,0.668164,0.420388,-1.179796,...,0.668917,0.714254,2.974603,-0.919589,-0.515430,-0.153111,-0.350218,-0.540962,4.983721,0
3,-0.994972,-0.440239,-0.178789,0.841250,-0.529808,0.033775,1.303832,0.175637,0.662489,-1.076888,...,-0.190105,-0.001408,-0.578786,-1.531963,0.944482,-0.298959,0.379163,0.382611,1.418291,0
4,-0.994960,-0.526089,0.579239,0.713861,0.265632,-0.270695,0.317183,0.491625,-0.546463,0.700808,...,0.048266,0.739092,-0.407980,0.126293,-0.333308,0.976221,1.347133,1.553716,0.670579,0


In [6]:
# Defining the labeled and unlabeled dataset classes
class LabeledDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __len__(self):
        return len(self.x)

    def __getitem__(self, index):
        vals = torch.tensor(self.x[index], dtype=torch.float32)
        label = torch.tensor(self.y[index], dtype=torch.float32)
        return vals, label

#### Dataset Splitting for Training and Testing

The original dataset is split into training and testing parts to evaluate the model's performance accurately. This step is crucial for validating the effectiveness of the training on unseen data.


In [7]:
from sklearn.model_selection import train_test_split

# Splitting the dataset into training and test sets for model evaluation
df_train, df_test = train_test_split(df, test_size=0.7, shuffle=True, random_state=42)
X_train, y_train = df_train.iloc[:, :-1].values, df_train.iloc[:, -1:].values
X_test, y_test = df_test.iloc[:, :-1].values, df_test.iloc[:, -1:].values

# Labeled datasets for supervised learning tasks
trainset = LabeledDataset(X_train, y_train)  # Corrected to include training data
testset = LabeledDataset(X_test, y_test)     # Test set with proper labels

# Printing the shapes of the datasets for verification
print(f"Labeled Trainset Shape: {len(trainset)}, {trainset.x.shape[1]}")
print(f"Labeled Testset Shape: {len(testset)}, {testset.x.shape[1]}")

Labeled Trainset Shape: 85442, 30
Labeled Testset Shape: 199365, 30


#### Initial Setup and Model Configuration

This section initializes the environment by setting a fixed random seed to ensure reproducibility of results. It imports necessary configurations and initializes model parameters with specific configurations. The model specified here is set to have no core model but uses a 'tabnet' encoder model for data processing, which is particularly tailored for structured or tabular data like credit card transactions.


In [8]:
# Set a fixed random seed for reproducibility of experiments
from nn.utils.init import set_random_seed
set_random_seed(0)

# Importing configuration setups for ML parameters and data
import torch
from tools.setting.ml_params import MLParameters
from tools.setting.data_config import DataConfig
from trainer_hub import TrainerHub

# Configuration for the data handling, defining dataset specifics and the task type
data_config = DataConfig(dataset_name='CreditCardFraudDetection', task_type='binary_classification', obs_shape=[n_features], label_size=n_classes)

# Initializing ML parameters without a core model and setting the encoder model to 'tabnet' with specific configurations
ml_params = MLParameters(core_model='tabnet', encoder_model='none')

# Setting training parameters and device configuration
ml_params.training.num_epoch = 10
ml_params.model.core_config.num_layers = 5

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

# Create a TrainerHub instance to manage training and data processing
trainer_hub = TrainerHub(ml_params, data_config, device, use_print=True, use_wandb=False)


c:\Users\CCNets-team\anaconda3\lib\site-packages\transformers\utils\generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [9]:
load_model = False
if load_model:
    trainer_hub.load_trainer(core_model = True)
else:
    trainer_hub.train(trainset)

Epochs:   0%|          | 0/10 [00:00<?, ?it/s]

Iterations:   0%|          | 0/1335 [00:00<?, ?it/s]

[0/10][50/1335][Time 8.27]
Unified LR across all optimizers: 0.0001995308238189185
--------------------Training Metrics--------------------
Cooperative Network(core):  Three Tabnet
Inf: 1.3385	Gen: 0.7746	Rec: 1.5001	E: 0.6130	R: 2.0640	P: 0.9362
[0/10][100/1335][Time 8.14]
Unified LR across all optimizers: 0.00019907191565870155
--------------------Training Metrics--------------------
Cooperative Network(core):  Three Tabnet
Inf: 1.2660	Gen: 0.7274	Rec: 1.4309	E: 0.5624	R: 1.9695	P: 0.8923
[0/10][150/1335][Time 7.96]
Unified LR across all optimizers: 0.00019861406295796434
--------------------Training Metrics--------------------
Cooperative Network(core):  Three Tabnet
Inf: 1.1247	Gen: 0.7519	Rec: 1.3875	E: 0.4891	R: 1.7603	P: 1.0146
[0/10][200/1335][Time 7.74]
Unified LR across all optimizers: 0.00019815726328921765
--------------------Training Metrics--------------------
Cooperative Network(core):  Three Tabnet
Inf: 0.7815	Gen: 0.7270	Rec: 1.1781	E: 0.3304	R: 1.2326	P: 1.1236
[0/10]

Iterations:   0%|          | 0/1335 [00:00<?, ?it/s]

[1/10][15/1335][Time 7.97]
Unified LR across all optimizers: 0.0001879360071406698
--------------------Training Metrics--------------------
Cooperative Network(core):  Three Tabnet
Inf: 0.3224	Gen: 0.7359	Rec: 0.8635	E: 0.1949	R: 0.4500	P: 1.2770
[1/10][65/1335][Time 7.47]
Unified LR across all optimizers: 0.000187503766318802
--------------------Training Metrics--------------------
Cooperative Network(core):  Three Tabnet
Inf: 0.3380	Gen: 0.7443	Rec: 0.8785	E: 0.2037	R: 0.4722	P: 1.2849
[1/10][115/1335][Time 7.81]
Unified LR across all optimizers: 0.00018707251962323787
--------------------Training Metrics--------------------
Cooperative Network(core):  Three Tabnet
Inf: 0.3412	Gen: 0.7518	Rec: 0.8917	E: 0.2014	R: 0.4811	P: 1.3023
[1/10][165/1335][Time 7.57]
Unified LR across all optimizers: 0.0001866422647675502
--------------------Training Metrics--------------------
Cooperative Network(core):  Three Tabnet
Inf: 0.3231	Gen: 0.7446	Rec: 0.8856	E: 0.1821	R: 0.4641	P: 1.3072
[1/10][215

In [ ]:
# DataLoader for processing training data in larger batches without shuffling
test_loader = torch.utils.data.DataLoader(dataset=testset, batch_size=256, shuffle=False, drop_last=False)

# CCNet setup for generating synthetic manipulated data
ccnet = trainer_hub.core_ccnet
manipulated_data = None
random_labels = None
original_labels = None
probability_positive = 0.0017 # 0.17% of the data is positive

# Generate synthetic data to enhance the diversity of the training dataset
for data, labels in test_loader:
    data = data.to(device)
    labels = labels.to(device)
    
    # Create random labels with 0.17% positive (1) and 99.83% negative (0) labels
    random_label = (torch.rand_like(labels) < probability_positive).float()

    # Use CCNet to explain the original data and generate synthetic counterparts
    explanations = ccnet.explain(data)
    synthetic_data = ccnet.produce(random_label, explanations)
    
    # Detach synthetic data and labels from GPU for further processing
    synthetic_data = synthetic_data.detach().cpu()
    random_label = random_label.detach().cpu()
    labels = labels.detach().cpu()

    # Accumulate the generated data for analysis and training
    if manipulated_data is None:
        manipulated_data = synthetic_data
        random_labels = random_label
        original_labels = labels
    else:
        manipulated_data = torch.cat([manipulated_data, synthetic_data], dim=0)
        random_labels = torch.cat([random_labels, random_label], dim=0)
        original_labels = torch.cat([original_labels, labels], dim=0)

# Output the shapes of the datasets for verification
print(f"Manipulated Data Shape: {manipulated_data.shape}")
print(f"Reversed Labels Shape: {random_labels.shape}")
print(f"Original Labels Shape: {original_labels.shape}")

# Create datasets for both synthetic and original label scenarios
random_label_testset = LabeledDataset(manipulated_data.numpy(), random_labels.numpy())
original_label_testset = LabeledDataset(manipulated_data.numpy(), original_labels.numpy())


#### Training Supervised Models

This section outlines the process of training supervised learning models using both original and synthetic datasets. The `train_supervised_model` function is designed to iterate through the dataset, perform forward passes, compute loss, and update model weights using backpropagation.


In [ ]:
def train_supervised_model(model, dataset, num_epoch=5):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
    # Initialize the optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    # Ensure reproducibility by resetting the random seed
    # Create DataLoader for batch processing
    trainloader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True)
    # Training loop
    for epoch in range(num_epoch):  # Train for 2 epochs as an example
        for i, (data, label) in enumerate(trainloader):
            data = data.to(device).clone().detach()
            label = label.to(device)
            # Perform forward pass
            output = model(data)
            # Compute loss
            loss = torch.nn.functional.binary_cross_entropy(output, label)
            # Backward pass to compute gradients
            loss.backward()
            # Update weights
            optimizer.step()
            # Reset gradients
            optimizer.zero_grad()


In [ ]:

class MLP(torch.nn.Module):
    def __init__(self, input_size, output_size, num_layers=4, hidden_size=256):
        super(MLP, self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_size = hidden_size
        
        # Create a list to hold all layers
        layers = []
        
        # Input layer
        layers.append(torch.nn.Linear(input_size, hidden_size))
        layers.append(torch.nn.ReLU())
        
        # Hidden layers
        for _ in range(num_layers - 2):
            layers.append(torch.nn.Linear(hidden_size, hidden_size))
            layers.append(torch.nn.ReLU())
        
        # Output layer
        layers.append(torch.nn.Linear(hidden_size, output_size))
        
        # Register all layers
        self.layers = torch.nn.Sequential(*layers)

    def forward(self, x):
        x = self.layers(x)
        return torch.sigmoid(x)
    
# Initialize and train a model on the recreated dataset
decision_making_model = MLP(input_size=n_features, output_size=n_classes).to(device)
train_supervised_model(decision_making_model, trainset)

#### Evaluating Model Performance


In [ ]:
from sklearn.metrics import f1_score

def get_f1_score(model, input_testset, device, batch_size=64):
    model.eval()  # Set the model to evaluation mode
    y_true = []
    y_pred = []
    # DataLoader for testing
    test_loader = torch.utils.data.DataLoader(input_testset, batch_size=batch_size, shuffle=False)

    # No gradient computation needed during inference
    with torch.no_grad():
        for data, label in test_loader:
            data = data.to(device)
            label = label.to(device)
            output = model(data)
            # Process output for binary classification
            predicted = (output.squeeze() > 0.5).long()
            y_true.extend(label.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())

    # Compute and return the F1 score
    score = f1_score(y_true, y_pred, average='binary')
    return score

# Calculate F1 scores for both models
fraud_detection_f1_score = get_f1_score(decision_making_model, testset, device)
random_testset_f1_score = get_f1_score(decision_making_model, random_label_testset, device)
manipulated_testset_f1_score = get_f1_score(decision_making_model, original_label_testset, device)

# Output the results
print("F1 score of the supervised learning model tested on the original data: ", fraud_detection_f1_score)
print("F1 score of the supervised learning model tested on the manipulated data with random label: ", random_testset_f1_score)
print("F1 score of the supervised learning model tested on the manipulated data with original label: ", manipulated_testset_f1_score)